In [7]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

Crearemos 3 variables:
1. `password`: la contraseña obtenida para su app seguridad
2. `origen`: correo que enviará la alerta
3. `destino`: destino de la alerta

In [8]:
password = ""
origen= ""
destino=""

# Creación de un servidor SMTP:
SMTP (Simple Mail Transfer Protocol) es un protocolo de comunicación utilizado para el envío de mensajes de correo electrónico entre servidores de correo y clientes de correo electrónico.
Un servidor SMTP es un software que se encarga de enviar, recibir y retransmitir correos electrónicos en una red. 

In [9]:
server = smtplib.SMTP('smtp.gmail.com: 587')
server.starttls()
server.login(origen, password)

(235, b'2.7.0 Accepted')

## Función para enviar un correo:

In [10]:
def enviar(origen, destino, objetos=1):
    mensaje = MIMEMultipart()
    mensaje['From'] = origen
    mensaje['To'] = destino
    mensaje['Subject'] = "Alerta de seguridad"
    # Agregar el cuerpo del mensaje
    cuerpo = f'ALERTA - Se han detectado {objetos} objetos !!'

    mensaje.attach(MIMEText(cuerpo, 'plain'))
    server.sendmail(origen,destino, mensaje.as_string())

## Creación de una clase para detectar objetos

In [11]:
class DeteccionObjetos:
    def __init__(self, capture_index):
        # default parameters
        self.capture_index = capture_index
        self.email_sent = False

        # model information
        self.model = YOLO("yolov8n.pt")

        # visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # device information
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def predict(self, im0):
        results = self.model(im0)
        return results

    def display_fps(self, im0):
        self.end_time = time()
        fps = 1 / np.round(self.end_time - self.start_time, 2)
        text = f'FPS: {int(fps)}'
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(im0, (20 - gap, 70 - text_size[1] - gap), (20 + text_size[0] + gap, 70 + gap), (255, 255, 255), -1)
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        class_ids = []
        self.annotator = Annotator(im0, 3, results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        names = results[0].names
        for box, cls in zip(boxes, clss):
            class_ids.append(cls)
            self.annotator.box_label(box, label=names[int(cls)], color=colors(int(cls), True))
        return im0, class_ids

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        frame_count = 0
        while True:
            self.start_time = time()
            ret, im0 = cap.read()
            assert ret
            results = self.predict(im0)
            im0, class_ids = self.plot_bboxes(results, im0)

            if len(class_ids) > 0:  # Only send email If not sent before
                if not self.email_sent:
                    enviar(origen, destino, len(class_ids))
                    self.email_sent = True
            else:
                self.email_sent = False

            self.display_fps(im0)
            cv2.imshow('YOLOv8 Detection', im0)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
        server.quit()



In [12]:
detector = DeteccionObjetos(capture_index=0)
detector()


0: 480x640 1 person, 1 chair, 85.0ms
Speed: 3.0ms preprocess, 85.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 73.0ms
Speed: 2.0ms preprocess, 73.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 70.0ms
Speed: 1.0ms preprocess, 70.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 73.0ms
Speed: 1.0ms preprocess, 73.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 1 couch, 70.0ms
Speed: 1.0ms preprocess, 70.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 71.0ms
Speed: 1.0ms preprocess, 71.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 65.0ms
Speed: 1.0ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 chair, 69.0ms
Speed: 1


KeyboardInterrupt

